In [59]:
import os
from os.path import isfile, join
from pathlib import Path
import numpy as np
from nibabel.testing import data_path
import nibabel as nib
import json
import pdb
import glob
from nilearn import plotting as nip
from nilearn import image
import pandas as pd

In [103]:

mypath = str(Path().absolute())
labelpath = join(mypath,'PyNets/Labels')
label_files = [f for f in os.listdir(labelpath) if os.path.isfile(join(labelpath,f))]
files = [f.split('.')[0] for f in label_files]
files = [i for i in files if not i.startswith("__")]
files = [i for i in files if not i.startswith("aal_c")]
files = [i for i in files if not i.startswith("Random")]
files = [i for i in files if not i.startswith("Voxel")]
files = files[6:len(files)-1]
files

['AALTzourioMazoyer2002',
 'AICHAreorderedJoliot2015',
 'AAL2zourioMazoyer2002',
 'AICHAJoliot2015',
 'MICCAI2012MultiAtlasLabelingWorkshopandChallengeNeuromorphometrics']

In [104]:
for f in files:
    print(f)
    try:
        filename = str(f)+'.txt'
        filename = join(labelpath,filename)
        df = pd.read_csv(filename, sep=' ', header=None, usecols=[0,1])
        df.columns = ["Region", "Label"]
    except ValueError:
        filename = str(f)+'.txt'
        filename = join(labelpath,filename)
        df = pd.read_csv(filename, sep='\t', header=None, usecols=[0,1])
        df.columns = ["Region", "Label"]
    outname = f+'.json'
    jsonpath = join(mypath,'PyNets/JSONs')
    outname = join(jsonpath,outname)
    createJSON(df,outname)
    
    niftipath = join(mypath,'PyNets/atlases/')
    niftiname = f+'.nii.gz'
    niftiname = join(niftipath,niftiname)
    Size_JSON(outname, niftiname, outname)
    

AALTzourioMazoyer2002
<class 'dict'>
<class 'dict'>
AICHAreorderedJoliot2015
<class 'dict'>
<class 'dict'>
AAL2zourioMazoyer2002
<class 'dict'>
<class 'dict'>
AICHAJoliot2015
<class 'dict'>
<class 'dict'>
MICCAI2012MultiAtlasLabelingWorkshopandChallengeNeuromorphometrics
<class 'dict'>
<class 'dict'>


In [94]:
def createJSON(df, outname):
    data = {}
    data['0'] = {}
    data['0']['label'] = 'empty'
    print(type(data))
    for row in range(1,len(df)+1):
        data[str(row)]={}
        data[str(row)]['label'] = df['Label'][row-1]
    with open(outname, 'w') as outfile:  
        json.dump(data, outfile, indent=4, sort_keys=True)

In [95]:
def Size_JSON(json_file, nifti_file, outname):    
    img = nib.load(nifti_file)
    img_data = img.get_fdata()
    img_flat = img_data.flatten()
    img_flat.tolist()
    img_flat = [int(i) for i in img_flat]
    values = np.unique(img_flat)
    counts = np.array(range(1, len(img_flat)+1))
    counts = [int(i) for i in counts]
    img_flatten = list(zip(img_flat, counts))
    img_flatten = sorted(img_flatten, key = lambda l:l[0])

    img_cell = [[] for i in range(len(values))]
    start = 0
    for i in range(0,len(values)):
        img_list = [x[0] for x in img_flatten]
        end = max(loc for loc, val in enumerate(img_list) if val == values[i])
        for j in range(start,end):
            img_cell[i].append(img_flatten[j][1])  
        start = end+1
    
    with open(json_file) as f:
        data = json.load(f)
        print(type(data))
    
    for i in range(0,len(values)):
        data[str(i)]["size"] = str(len(img_cell[i]))
        #val = str(len(img_cell[i]))
        #print(val)
        #data[str(i)].append({
        #    'size': str(len(img_cell[i]))
        #})
    
    with open(outname, 'w') as f:
        json.dump(data, f, indent=4, sort_keys=True)